# Lib

In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 10000)


import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
import sys
import os
import glob

if sys.platform=='win32':
    sys.path.insert(0,".\..\src")
        
elif sys.platform=='linux':
    sys.path.insert(0,"./../src")

from utils import utils_ml, parameters


In [2]:
if sys.platform=='win32':
    !dir .\
        
elif sys.platform=='linux':
    !ls ./

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks

16/11/2022  03:42    <DIR>          .
15/11/2022  21:08    <DIR>          ..
10/09/2022  23:28    <DIR>          .ipynb_checkpoints
16/11/2022  03:42         1.047.170 1.EA.ipynb
16/11/2022  03:38         1.047.170 2.Feature_engineering.ipynb
               2 arquivo(s)      2.094.340 bytes
               3 pasta(s)   362.633.310.208 bytes dispon�veis


In [3]:
!dir .\..\src

 O volume na unidade C � OS
 O N�mero de S�rie do Volume � C498-8BF5

 Pasta de c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\src

15/11/2022  20:32    <DIR>          .
15/11/2022  21:08    <DIR>          ..
15/11/2022  20:43    <DIR>          features
15/11/2022  21:00    <DIR>          utils
11/09/2022  00:21    <DIR>          __pycache__
               0 arquivo(s)              0 bytes
               5 pasta(s)   362.633.244.672 bytes dispon�veis


# Reading

In [4]:
## para saber en que directório uno se encuentra
#
if sys.platform=='win32':
    !echo %cd%
        
elif sys.platform=='linux':
    !pwd

c:\Users\caanp\OneDrive\Documents\repositories\ML-anti-fraud\notebooks


In [149]:
if sys.platform=='win32':
    df = pd.read_csv('.\..\data\dados.csv')
        
elif sys.platform=='linux':
    df = pd.read_csv('./../data/dados.csv')


## dropping column "o" becaise it has 78% missing. and this is a boolean columns
# df.drop("o", axis=1, inplace=True)

# EA

In [150]:
df.rename(columns={"a":"a_int",
                    "p":"p_boolean","n":'n_boolean',
                     "k":"k_float","e":"e_float","h":"h_float","l":"l_float","f":"f_float","m":"m_float","m":"m_float","b":"b_float","c":"c_float","d":"d_float","o":"o_obj",
                     'i': 'produto','j':'categoria_produto','g':'pais'}
       , inplace = True)

## Transform categorical features
df['n_boolean'] = df['n_boolean'].astype("category")
df['p_boolean'] = df['p_boolean'].astype("category")


## creating columns os date and hour
df['data'] = df['fecha'].apply(lambda x: x[:10])
df['hora'] = df['fecha'].apply(lambda x: x[10:])

# WoE transformation

In [151]:
cats_cols = utils_ml.list_subset_words( df.columns.tolist() , ["bool","cat",'obj','pais'])
numeric_cols = [x for x in df.columns.tolist() if df[x].dtype!="O" and df[x].dtype!="category" and x!="fraude" ]

features = cats_cols + numeric_cols
features

['n_boolean',
 'p_boolean',
 'categoria_produto',
 'o_obj',
 'pais',
 'a_int',
 'b_float',
 'c_float',
 'd_float',
 'e_float',
 'f_float',
 'h_float',
 'k_float',
 'l_float',
 'm_float',
 'monto',
 'score']

In [152]:
dff = df.copy()

for variable in features:

    label = "fraude"
    dtype_ = 'categorical' if df[variable].dtype=='O' or df[variable].dtype=='category' else 'numerical'
    monotonic = True
    save_path_pkl = '.\..\src\\features\\'


    df = utils_ml.encode_woe_var(df, variable, label, dtype_, monotonic = True, save_path_pkl = save_path_pkl )

In [153]:
features_bins_woe = utils_ml.list_subset_words( df.columns.tolist(), ["bin"])
features_bins_woe

['n_boolean_bins',
 'p_boolean_bins',
 'categoria_produto_bins',
 'o_obj_bins',
 'pais_bins',
 'a_int_bins',
 'b_float_bins',
 'c_float_bins',
 'd_float_bins',
 'e_float_bins',
 'f_float_bins',
 'h_float_bins',
 'k_float_bins',
 'l_float_bins',
 'm_float_bins',
 'monto_bins',
 'score_bins']

In [154]:
len(features_bins_woe)

17

# Model

In [155]:
df_2_model = df[features_bins_woe + ["fraude"]]

# df_2_model_transform = df_2_model.select_dtypes(exclude=['number']) \
#                 .apply(LabelEncoder().fit_transform) \
#                 .join(df_2_model.select_dtypes(include=['number']))


In [156]:
cat_vars = features_bins_woe
df_2_model_dum = df_2_model.copy()

for var in cat_vars:

    cat_list='var'+'_'+var
    cat_list = pd.get_dummies( df_2_model_dum[var] , prefix=var)
    data_dum = df_2_model_dum.join(cat_list)
    df_2_model_dum = data_dum


data_vars = df_2_model_dum.columns.tolist()
to_keep = [i for i in data_vars if i not in cat_vars]

df_2_model_dum = df_2_model_dum[to_keep]


In [174]:
from sklearn.model_selection import train_test_split

X = df_2_model_dum.drop(['fraude'], axis=1)
y = df_2_model_dum['fraude']

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=0, stratify=y )

## Logistic regression

In [176]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

clf = LogisticRegression(random_state=0, fit_intercept=True, penalty='l2',class_weight={0: 1, 1: 95} )

model = clf.fit(X_train, y_train )

c:\Users\caanp\miniconda3\envs\venv_ml\lib\site-packages\sklearn\linear_model\_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [177]:
df_2_model['probabilty'] = model.predict_proba(X)[:,1]

C:\Users\caanp\AppData\Local\Temp\ipykernel_22028\2283094811.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_2_model['probabilty'] = model.predict_proba(X)[:,1]


In [178]:
df_ivs_ls = []

for variable in dff.columns.tolist():

    label = "fraude"
    dtype_ = 'categorical' if dff[variable].dtype=='O' or dff[variable].dtype=='category' else 'numerical'

    df_iv = utils_ml.WOE_IV(df = dff , variable = variable, dtype_ = dtype_ , label = label, monotonic= True)[[ 'variable' , 'IV_var', 'KS']].drop_duplicates()

    df_ivs_ls.append(df_iv)

pd.concat(df_ivs_ls).sort_values("IV_var", ascending=False)

,variable,IV_var,KS
0,fecha,12.113640,99.544561
0,produto,6.369097,88.306667
0,hora,4.346361,84.314386
0,categoria_produto,1.641776,39.766316
0,o_obj,1.094238,44.994386
0,score,1.021894,43.152982
0,f_float,0.447805,27.785965
0,l_float,0.439939,25.305965
0,n_boolean,0.361450,22.844211
0,m_float,0.337380,22.929825


In [182]:
coef_df = pd.DataFrame( {"Cols": model.feature_names_in_,  "coef": model.coef_[0] })
# coef_df.sort_values("coef", ascending=False)

In [180]:
probability = "score"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[0, 8]",417,14583,7500,142500,0,0,5.560000,10.233684,5.560000,10.233684,42.42807,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[8, 18]",694,14306,7083,127917,14583,417,9.253333,10.039298,14.813333,20.272982,42.42807,0.944400,0.052467,0.102337,0.099411,0.214636
2,15000,"[18, 28]",492,14508,6389,113611,28889,1111,6.560000,10.181053,21.373333,30.454035,42.42807,0.851867,0.053242,0.202730,0.100220,0.212967
3,15000,"[28, 38]",45,14955,5897,99103,43397,1603,0.600000,10.494737,21.973333,40.948772,42.42807,0.786267,0.056162,0.304540,0.104836,0.218407
4,15000,"[38, 48]",91,14909,5852,84148,58352,1648,1.213333,10.462456,23.186667,51.411228,42.42807,0.780267,0.065022,0.409488,0.120041,0.243833
5,15000,"[48, 58]",195,14805,5761,69239,73261,1739,2.600000,10.389474,25.786667,61.800702,42.42807,0.768133,0.076813,0.514112,0.139661,0.274333
6,15000,"[58, 68]",293,14707,5566,54434,88066,1934,3.906667,10.320702,29.693333,72.121404,42.42807,0.742133,0.092767,0.618007,0.164919,0.309222
7,15000,"[68, 78]",783,14217,5273,39727,102773,2227,10.440000,9.976842,40.133333,82.098246,42.42807,0.703067,0.117178,0.721214,0.200876,0.351533
8,15000,"[78, 88]",1295,13705,4490,25510,116990,3010,17.266667,9.617544,57.400000,91.715789,42.42807,0.598667,0.149667,0.820982,0.239467,0.374167


In [181]:
probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_2_model, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,15000,"[0.0009367637163490618, 0.013070802023719068]",1,14999,7500,142500,0,0,0.013333,10.525614,0.013333,10.525614,63.22807,1.000000,0.050000,0.000000,0.095238,0.208333
1,15000,"[0.01307193788613293, 0.06532272328231824]",6,14994,7499,127501,14999,1,0.080000,10.522105,0.093333,21.047719,63.22807,0.999867,0.055548,0.105256,0.105249,0.227242
2,15000,"[0.06534163692084798, 0.27443018369292016]",38,14962,7493,112507,29993,7,0.506667,10.499649,0.600000,31.547368,63.22807,0.999067,0.062442,0.210477,0.117537,0.249767
3,15000,"[0.27443414038487846, 0.3987155210045663]",81,14919,7455,97545,44955,45,1.080000,10.469474,1.680000,42.016842,63.22807,0.994000,0.071000,0.315474,0.132533,0.276111
4,15000,"[0.3987185122116206, 0.502139071602443]",124,14876,7374,82626,59874,126,1.653333,10.439298,3.333333,52.456140,63.22807,0.983200,0.081933,0.420168,0.151262,0.307250
5,15000,"[0.502139845426772, 0.6127689545764381]",206,14794,7250,67750,74750,250,2.746667,10.381754,6.080000,62.837895,63.22807,0.966667,0.096667,0.524561,0.175758,0.345238
6,15000,"[0.6127719627882745, 0.7400562482227317]",364,14636,7044,52956,89544,456,4.853333,10.270877,10.933333,73.108772,63.22807,0.939200,0.117400,0.628379,0.208711,0.391333
7,15000,"[0.7400582682275161, 0.851131540855908]",675,14325,6680,38320,104180,820,9.000000,10.052632,19.933333,83.161404,63.22807,0.890667,0.148444,0.731088,0.254476,0.445333
8,15000,"[0.8511347551807305, 0.933017252927653]",1250,13750,6005,23995,118505,1495,16.666667,9.649123,36.600000,92.810526,63.22807,0.800667,0.200167,0.831614,0.320267,0.500417


In [ ]:
df_2_model['probabilty'] = model.predict_proba(X)[:,1]

In [190]:
df_test = pd.DataFrame()
df_test['fraude'] = y_test
df_test['probabilty'] = model.predict_proba(X_test)[:,1]

probability = "probabilty"
label="fraude"
quantil=10

utils_ml.metric_evaluation(df_test, probability, label, quantil)

,total,faixa_prob,total_um,total_zero,TP,FP,TN,FN,perc_um,perc_zero,csum_um,csum_zero,ks,recall,precision,specificity,F1,F2
quantil,,,,,,,,,,,,,,,,,,
0,4500,"[0.0009670005026312834, 0.013266370127421953]",0,4500,2250,42750,0,0,0.000000,10.526316,0.000000,10.526316,62.45614,1.000000,0.050000,0.000000,0.095238,0.208333
1,4500,"[0.01326791821269253, 0.06815549535447887]",4,4496,2250,38250,4500,0,0.177778,10.516959,0.177778,21.043275,62.45614,1.000000,0.055556,0.105263,0.105263,0.227273
2,4500,"[0.06819659244201087, 0.2800081168920057]",20,4480,2246,33754,8996,4,0.888889,10.479532,1.066667,31.522807,62.45614,0.998222,0.062389,0.210433,0.117438,0.249556
3,4500,"[0.28001791033340123, 0.4003112056544835]",30,4470,2226,29274,13476,24,1.333333,10.456140,2.400000,41.978947,62.45614,0.989333,0.070667,0.315228,0.131911,0.274815
4,4500,"[0.4004099071388192, 0.5049574579364107]",40,4460,2196,24804,17946,54,1.777778,10.432749,4.177778,52.411696,62.45614,0.976000,0.081333,0.419789,0.150154,0.305000
5,4500,"[0.505007326563319, 0.6179522110028688]",60,4440,2156,20344,22406,94,2.666667,10.385965,6.844444,62.797661,62.45614,0.958222,0.095822,0.524117,0.174222,0.342222
6,4500,"[0.6179551945176108, 0.745272245500609]",112,4388,2096,15904,26846,154,4.977778,10.264327,11.822222,73.061988,62.45614,0.931556,0.116444,0.627977,0.207012,0.388148
7,4500,"[0.745340888027975, 0.851427142738993]",199,4301,1984,11516,31234,266,8.844444,10.060819,20.666667,83.122807,62.45614,0.881778,0.146963,0.730620,0.251937,0.440889
8,4500,"[0.8514694918985806, 0.9333543975438213]",374,4126,1785,7215,35535,465,16.622222,9.651462,37.288889,92.774269,62.45614,0.793333,0.198333,0.831228,0.317333,0.495833
